In [ ]:
import matplotlib.pyplot as plt


def roc_plot(y_test, y_pred):
    from sklearn.metrics import roc_curve, auc
    from sklearn.preprocessing import label_binarize
    from itertools import cycle

    hand_name = {
        0: 'Nothing in hand',
        1: 'One pair',
        2: 'Two pairs',
        3: 'Three of a kind',
        4: 'Straight',
        5: 'Flush',
        6: 'Full house',
        7: 'Four of a kind',
        8: 'Straight flush',
        9: 'Royal flush',
    }

    y_test = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    n_classes = y_test.shape[1]

    fpr, tpr, roc_auc = dict(), dict(), dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    colors = cycle(['red', 'purple', 'green', 'orange', 'blue',
                   'brown', 'pink', 'gray', 'olive', 'cyan'])
    plt.figure(figsize=(20, 20))
    for i, color in zip(range(n_classes), colors):
        plt.subplot(4, 3, i + 1)
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'{i}. {hand_name[i]} (area = {roc_auc[i]:.2f})')

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([-.05, 1.05])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Poker Hands ROC')
        plt.legend(loc='lower right')
    
    plt.tight_layout()
    plt.show(block=True)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

df = pd.read_csv('./data/SIRTUIN6.csv')

x = normalize(df.iloc[:, :6].values)
y = df['Class'].values

print(x)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=.2, stratify=y, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=.25, stratify=y_train, random_state=42)

cat_model = CatBoostClassifier(task_type='GPU', verbose=200)

cat_model.fit(x_train, y_train, eval_set=(x_val, y_val))

print(cat_model.get_feature_importance())

y_pred = cat_model.predict(x_test)
y_prob = cat_model.predict_proba(x_test)

In [ ]:
from sklearn.metrics import classification_report

roc_plot(y_test, y_prob)
print(classification_report(y_test, y_pred, digits=5, zero_division=1))